# Multi-Source Document Processing & Knowledge Extraction

This notebook implements the core GraphRAG pipeline from Microsoft's paper ["From Local to Global: A Graph RAG Approach"](https://arxiv.org/abs/2404.16130).

## Pipeline Steps
1. **Define and fetch sources** - 7 documents from arXiv + web (with offline fallbacks)
2. **Chunk all documents** - Split into 600-token chunks with 100-token overlap
3. **Entity extraction** - Use LLM to identify named entities per document
4. **Relationship extraction** - Extract connections between entities
5. **Claims extraction** - Extract factual statements about entities
6. **Cross-document merge** - Deduplicate and merge entities across all sources

## Model
Using locally deployed Ollama with `qwen2.5:3b`

## Sources
- 4 arXiv papers (AI, biology, climate, astrophysics)
- 3 web articles (neuroscience, economics, space exploration)

## Setup

In [18]:
import httpx
import json
import time
import tempfile
from typing import Any
from datetime import datetime, timezone
from dataclasses import dataclass, field, asdict
from langchain_text_splitters import RecursiveCharacterTextSplitter

import arxiv
import pymupdf
import trafilatura

OLLAMA_BASE_URL = "http://localhost:11434"
MODEL = "qwen2.5:3b"

In [19]:
# Verify Ollama is running
response = httpx.get(f"{OLLAMA_BASE_URL}/api/tags")
models = [m["name"] for m in response.json().get("models", [])]
print(f"Available models: {models}")
assert MODEL in models, f"Model {MODEL} not found. Please run: ollama pull {MODEL}"

Available models: ['nomic-embed-text:latest', 'qwen2.5:3b']


## Step 1: Define and Fetch Sources

We fetch 7 documents from diverse domains to stress-test the extraction pipeline. Each source has hardcoded fallback text so the notebook runs offline.

In [20]:
# --- Source limits (for debugging/testing) ---
# Set to None to use all sources, or a number to limit.
# Example: ARXIV_LIMIT=1, WEB_LIMIT=0 → only the first arXiv paper
ARXIV_LIMIT = 1   # None = all 4, or 1/2/3
WEB_LIMIT = 0     # None = all 3, or 0/1/2

@dataclass
class SourceDocument:
    source_id: str       # e.g. "arxiv:2404.16130"
    source_type: str     # "arxiv" or "web"
    title: str
    url: str
    content: str
    content_type: str    # "research_paper", "news", "reference"
    fetched_at: str = ""

    def __post_init__(self):
        if not self.fetched_at:
            self.fetched_at = datetime.now(timezone.utc).isoformat()

# --- Source configurations ---

ARXIV_SOURCES = [
    {"id": "2404.16130", "content_type": "research_paper",
     "title": "From Local to Global: A Graph RAG Approach to Query-Focused Summarization"},
    {"id": "2404.18021", "content_type": "research_paper",
     "title": "CRISPR-GPT: An LLM Agent for Automated Design of Gene-Editing Experiments"},
    {"id": "2312.14090", "content_type": "research_paper",
     "title": "Climate Economics and Finance: A Review"},
    {"id": "2304.04869", "content_type": "research_paper",
     "title": "The JWST Mission: Astrophysics Enabled"},
]

WEB_SOURCES = [
    {"url": "https://www.quantamagazine.org/memory-and-perception-are-intertwined-20240416/",
     "source_id": "web:quanta-memory", "content_type": "news",
     "title": "Memory and Perception Are Intertwined in the Brain"},
    {"url": "https://www.imf.org/en/Blogs/Articles/2024/01/14/ai-will-transform-the-global-economy-lets-make-sure-it-benefits-humanity",
     "source_id": "web:imf-ai-economy", "content_type": "news",
     "title": "AI Will Transform the Global Economy"},
    {"url": "https://www.planetary.org/space-missions/voyager",
     "source_id": "web:planetary-voyager", "content_type": "reference",
     "title": "Voyager: The Grand Tour of the Solar System"},
]

# Apply source limits
_arxiv_configs = ARXIV_SOURCES[:ARXIV_LIMIT] if ARXIV_LIMIT is not None else ARXIV_SOURCES
_web_configs = WEB_SOURCES[:WEB_LIMIT] if WEB_LIMIT is not None else WEB_SOURCES

print(f"Source limits: ARXIV_LIMIT={ARXIV_LIMIT}, WEB_LIMIT={WEB_LIMIT}")
print(f"  Using {len(_arxiv_configs)}/{len(ARXIV_SOURCES)} arXiv sources, {len(_web_configs)}/{len(WEB_SOURCES)} web sources")

# --- Hardcoded fallback content (offline mode) ---

FALLBACK_CONTENT = {
    "arxiv:2404.16130": """From Local to Global: A Graph RAG Approach to Query-Focused Summarization.
The use of retrieval-augmented generation (RAG) to retrieve relevant information from an external knowledge source enables large language models (LLMs) to answer questions over private or previously unseen document collections. However, RAG fails on global questions directed at an entire text corpus, such as "What are the main themes in the dataset?", since this is inherently a query-focused summarization (QFS) task. Prior QFS methods fail to scale to the quantities of text indexed by typical RAG systems. To combine the strengths of these contrasting methods, we propose a Graph RAG approach to question answering over private text corpora that scales with both the generality of user questions and the quantity of source text. Our approach uses an LLM to build a graph-based text index in two stages: first to derive an entity knowledge graph from the source documents, then to pre-generate community summaries for all groups of closely-related entities. Given a question, each community summary is used to generate a partial response, before all partial responses are summarized into a final response. For a class of global sensemaking questions over datasets in the 1 million token range, we show that Graph RAG leads to substantial improvements over a naive RAG baseline for both the comprehensiveness and diversity of generated answers.""",

    "arxiv:2404.18021": """CRISPR-GPT: An LLM Agent for Automated Design of Gene-Editing Experiments.
The integration of large language models (LLMs) with biological research holds significant promise for accelerating scientific discovery. CRISPR-GPT is a novel LLM agent that automates the design of CRISPR gene-editing experiments. It combines domain-specific knowledge of CRISPR biology with the reasoning capabilities of GPT-4 to guide researchers through the entire experimental workflow—from target gene selection and guide RNA design to delivery method optimization and off-target analysis. CRISPR-GPT integrates multiple bioinformatics tools, including sequence alignment algorithms, off-target prediction models, and primer design software, enabling end-to-end experimental planning. Evaluation across diverse gene-editing tasks demonstrates that CRISPR-GPT matches the performance of expert human researchers while significantly reducing experiment design time. The system handles multiple CRISPR platforms including Cas9, Cas12a, and base editors, adapting its recommendations based on the specific biological context. This work demonstrates the potential of AI-assisted scientific research and raises important questions about the responsible development of autonomous biological research agents.""",

    "arxiv:2312.14090": """Climate Economics and Finance: A Comprehensive Review.
Climate change poses fundamental challenges to economic systems worldwide. This review synthesizes research at the intersection of climate science, economics, and finance, covering carbon pricing mechanisms, green bond markets, stranded asset risks, and the economic modeling of climate damages. Integrated assessment models (IAMs) such as DICE and FUND translate climate projections into economic impacts, estimating the social cost of carbon (SCC) at $50-200 per ton. Carbon markets like the EU Emissions Trading System (EU ETS) and California's cap-and-trade program demonstrate market-based approaches to emissions reduction. The green bond market has grown from $3 billion in 2012 to over $500 billion in 2023, financing renewable energy, energy efficiency, and sustainable infrastructure. Central banks, including the European Central Bank (ECB) and Bank of England, are conducting climate stress tests on financial institutions to assess systemic risk. Physical climate risks—extreme weather, sea-level rise, and agricultural disruption—threaten $43 trillion in global assets by 2100. Transition risks from policy changes and technological shifts could render fossil fuel reserves worth $1-4 trillion as stranded assets. This review identifies carbon border adjustment mechanisms, climate-related financial disclosure frameworks such as TCFD, and natural capital accounting as critical frontiers in climate economics.""",

    "arxiv:2304.04869": """The James Webb Space Telescope Mission: Scientific Capabilities and Early Results.
The James Webb Space Telescope (JWST), launched in December 2021 on an Ariane 5 rocket, represents the most ambitious space observatory ever built. With its 6.5-meter primary mirror composed of 18 gold-coated beryllium segments, JWST operates at the second Lagrange point (L2), 1.5 million kilometers from Earth. The telescope carries four science instruments: NIRCam (Near-Infrared Camera), NIRSpec (Near-Infrared Spectrograph), MIRI (Mid-Infrared Instrument), and FGS/NIRISS (Fine Guidance Sensor/Near Infrared Imager and Slitless Spectrograph). Early science results have been transformative across multiple astrophysics domains. In exoplanet science, JWST detected carbon dioxide in the atmosphere of WASP-39b, a gas giant 700 light-years away. Deep field observations revealed galaxies forming just 300 million years after the Big Bang, challenging models of early galaxy formation. JWST has captured unprecedented images of the Carina Nebula, revealing star-forming regions previously hidden by dust. The telescope's sensitivity enables detailed spectroscopy of stellar nurseries, providing insights into the chemical composition of protoplanetary disks. The mission, led by NASA in partnership with ESA and CSA, has an expected operational lifetime of 10-20 years, far exceeding the original 5-year design goal.""",

    "web:quanta-memory": """Memory and Perception Are Deeply Intertwined in the Brain.
Neuroscience research reveals that the boundary between memory and perception is far more blurred than previously believed. The hippocampus, long considered the brain's memory center, plays an active role in perception. Studies show that the hippocampus predicts what we are about to see based on past experience, creating an internal model that shapes conscious perception in real time. Researchers at University College London used fMRI to demonstrate that hippocampal activity precedes visual cortex activation during familiar scene recognition, suggesting the brain uses memory templates to "pre-render" expected visual information. This predictive coding framework has implications for understanding Alzheimer's disease, where the breakdown of hippocampal prediction circuits may explain why patients experience perceptual disturbances before overt memory loss. The prefrontal cortex mediates between memory-based predictions and incoming sensory data, resolving conflicts when reality differs from expectation. Similar predictive mechanisms have been found in the auditory system, where the hippocampus anticipates sounds in familiar sequences. These findings challenge the traditional cognitive architecture that separates perception, memory, and prediction into distinct modules, instead suggesting a unified system where the brain constantly generates and tests predictions about the world using stored knowledge.""",

    "web:imf-ai-economy": """AI Will Transform the Global Economy — Let's Make Sure It Benefits Humanity.
The International Monetary Fund analysis indicates that artificial intelligence will affect approximately 40 percent of all jobs worldwide. In advanced economies, the exposure rises to 60 percent of jobs, where AI could either complement human workers—boosting productivity by 15-25 percent in affected sectors—or replace them entirely. The IMF proposes a comprehensive AI Preparedness Index measuring countries across digital infrastructure, human capital, labor market policies, and regulatory frameworks. Nordic countries, Singapore, and the United States score highest on AI readiness. Emerging markets and developing economies face a different challenge: while less immediately exposed to AI displacement (26 percent of jobs affected), they lack the infrastructure and skilled workforce to capture AI's productivity benefits, potentially widening the global inequality gap. The IMF recommends establishing social safety nets calibrated to the pace of AI adoption, investing in education programs that emphasize skills complementary to AI—such as critical thinking, creativity, and emotional intelligence—and creating international frameworks for AI governance. The analysis warns that without proactive policies, AI could increase income inequality within countries by up to 15 percentage points over the next decade, as high-skilled workers who effectively leverage AI tools see disproportionate wage gains.""",

    "web:planetary-voyager": """Voyager: The Grand Tour of the Solar System.
The Voyager program, launched by NASA in 1977, represents one of humanity's greatest exploration achievements. Voyager 1 and Voyager 2 were designed to take advantage of a rare planetary alignment occurring once every 176 years, enabling gravity-assist flybys of the outer planets. Voyager 1 visited Jupiter in 1979 and Saturn in 1980, discovering active volcanoes on Jupiter's moon Io—the first found beyond Earth—and the complex structure of Saturn's rings. Voyager 2 is the only spacecraft to have visited Uranus (1986) and Neptune (1989), revealing Uranus's extreme axial tilt and Neptune's Great Dark Spot. The spacecraft carry the Golden Record, a 12-inch gold-plated copper disc containing sounds and images representing life and culture on Earth, curated by a team led by Carl Sagan. Voyager 1 entered interstellar space in August 2012, becoming the first human-made object to leave the heliosphere, at a distance of 121 astronomical units from the Sun. Voyager 2 followed in November 2018. Both spacecraft continue to transmit scientific data using their 23-watt radio transmitters—roughly the power of a refrigerator light bulb—communicating with NASA's Deep Space Network. As of 2024, Voyager 1 is over 24 billion kilometers from Earth, traveling at 17 kilometers per second. The mission, originally designed for 5 years, has operated for over 46 years, with power from their radioisotope thermoelectric generators expected to sustain limited operations until approximately 2030.""",
}

# --- PDF text extraction ---

def extract_pdf_text(pdf_path: str) -> str:
    """Extract text from a PDF file using pymupdf."""
    doc = pymupdf.open(pdf_path)
    text_parts = []
    for page in doc:
        text_parts.append(page.get_text())
    doc.close()
    return "\n".join(text_parts)

# --- Fetch functions ---

def fetch_arxiv_sources(configs: list[dict]) -> list[SourceDocument]:
    """Fetch full paper content from arXiv PDFs. Falls back to abstract, then hardcoded content."""
    documents = []
    for cfg in configs:
        paper_id = cfg["id"]
        source_id = f"arxiv:{paper_id}"
        fallback = FALLBACK_CONTENT.get(source_id, "")

        try:
            client = arxiv.Client()
            search = arxiv.Search(id_list=[paper_id])
            results = list(client.results(search))
            if results:
                paper = results[0]
                title = paper.title
                url = paper.entry_id

                # Download and extract full PDF text
                with tempfile.TemporaryDirectory() as tmpdir:
                    pdf_path = paper.download_pdf(dirpath=tmpdir)
                    content = extract_pdf_text(pdf_path)

                if len(content) < 500:
                    raise ValueError(f"PDF text too short ({len(content)} chars), falling back to abstract")

                print(f"  [OK] {source_id}: fetched full PDF from arXiv ({len(content)} chars)")
            else:
                raise ValueError("No results")
        except Exception as e:
            print(f"  [FALLBACK] {source_id}: {e}")
            content = fallback
            title = cfg.get("title", paper_id)
            url = f"https://arxiv.org/abs/{paper_id}"

        documents.append(SourceDocument(
            source_id=source_id,
            source_type="arxiv",
            title=title,
            url=url,
            content=content,
            content_type=cfg["content_type"],
        ))
    return documents


def fetch_web_sources(configs: list[dict]) -> list[SourceDocument]:
    """Fetch article text from web pages. Falls back to hardcoded content."""
    documents = []
    for cfg in configs:
        source_id = cfg["source_id"]
        fallback = FALLBACK_CONTENT.get(source_id, "")

        try:
            downloaded = trafilatura.fetch_url(cfg["url"])
            if downloaded:
                content = trafilatura.extract(downloaded)
                if content and len(content) > 200:
                    print(f"  [OK] {source_id}: fetched from web ({len(content)} chars)")
                else:
                    raise ValueError("Extracted content too short")
            else:
                raise ValueError("Download failed")
        except Exception as e:
            print(f"  [FALLBACK] {source_id}: {e}")
            content = fallback

        documents.append(SourceDocument(
            source_id=source_id,
            source_type="web",
            title=cfg["title"],
            url=cfg["url"],
            content=content,
            content_type=cfg["content_type"],
        ))
    return documents


# --- Fetch all sources ---
print(f"\nFetching arXiv sources (downloading full PDFs)...")
arxiv_docs = fetch_arxiv_sources(_arxiv_configs)

print(f"\nFetching web sources...")
web_docs = fetch_web_sources(_web_configs)

all_documents = arxiv_docs + web_docs

print(f"\n{'='*60}")
print(f"SOURCES LOADED: {len(all_documents)}")
print(f"{'='*60}")
total_chars = 0
for doc in all_documents:
    print(f"  [{doc.source_type}] {doc.source_id}: {doc.title[:60]}")
    print(f"         {len(doc.content)} chars | {doc.content_type}")
    total_chars += len(doc.content)
print(f"\nTotal content: {total_chars:,} characters across {len(all_documents)} sources")

Source limits: ARXIV_LIMIT=1, WEB_LIMIT=0
  Using 1/4 arXiv sources, 0/3 web sources

Fetching arXiv sources (downloading full PDFs)...
  [OK] arxiv:2404.16130: fetched full PDF from arXiv (89608 chars)

Fetching web sources...

SOURCES LOADED: 1
  [arxiv] arxiv:2404.16130: From Local to Global: A Graph RAG Approach to Query-Focused 
         89608 chars | research_paper

Total content: 89,608 characters across 1 sources


## Step 2: Chunk All Documents

Following GraphRAG methodology: ~600 tokens per chunk with 100 token overlap.
Using character-based approximation (1 token ≈ 4 characters).
Each document is chunked independently.

In [21]:
# GraphRAG uses 600 tokens with 100 token overlap
CHUNK_SIZE = 600
CHUNK_OVERLAP = 100

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

# Chunk each document independently
source_chunks: dict[str, list[str]] = {}
total_chunks = 0

for doc in all_documents:
    doc_chunks = text_splitter.split_text(doc.content)
    source_chunks[doc.source_id] = doc_chunks
    total_chunks += len(doc_chunks)
    print(f"  {doc.source_id}: {len(doc_chunks)} chunks ({len(doc.content)} chars)")

print(f"\n{'='*40}")
print(f"Total: {total_chunks} chunks across {len(all_documents)} sources")

  arxiv:2404.16130: 193 chunks (89608 chars)

Total: 193 chunks across 1 sources


## Helper: Ollama Chat Function

In [22]:
MAX_RETRIES = 2
OLLAMA_TIMEOUT = 180.0  # seconds per request

def chat_ollama(prompt: str, system: str = "", temperature: float = 0.0) -> str:
    """Send a chat request to Ollama with retry logic."""
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            response = httpx.post(
                f"{OLLAMA_BASE_URL}/api/chat",
                json={
                    "model": MODEL,
                    "messages": messages,
                    "stream": False,
                    "options": {"temperature": temperature}
                },
                timeout=OLLAMA_TIMEOUT,
            )
            response.raise_for_status()
            return response.json()["message"]["content"]
        except (httpx.TimeoutException, httpx.HTTPStatusError) as e:
            if attempt < MAX_RETRIES:
                wait = 2 * attempt
                print(f"[retry {attempt}/{MAX_RETRIES}: {type(e).__name__}, waiting {wait}s] ", end="")
                time.sleep(wait)
            else:
                raise

# Test the connection
test_response = chat_ollama("Say 'Hello GraphRAG!' and nothing else.")
print(f"Ollama test: {test_response}")
print(f"Config: timeout={OLLAMA_TIMEOUT}s, max_retries={MAX_RETRIES}")

Ollama test: Hello GraphRAG!
Config: timeout=180.0s, max_retries=2


## Step 3: Entity, Relationship & Claims Extraction

Extract named entities, relationships, and claims from each document.

**Entity types:** PERSON, ORGANIZATION, LOCATION, EVENT, PRODUCT, DATE, MONEY, CONCEPT

In [23]:
@dataclass
class Entity:
    name: str
    type: str
    description: str
    source_chunk: int = 0

@dataclass
class Relationship:
    source: str
    target: str
    description: str
    strength: float = 1.0
    source_chunk: int = 0

@dataclass
class Claim:
    subject: str
    claim_type: str
    description: str
    date: str = ""
    source_chunk: int = 0


# --- Extraction prompts ---

ENTITY_EXTRACTION_PROMPT = """
You are an expert at extracting named entities from text.

Extract all named entities from the following text. For each entity provide:
1. name: The entity name (use UPPERCASE for consistency)
2. type: One of [PERSON, ORGANIZATION, LOCATION, EVENT, PRODUCT, DATE, MONEY, CONCEPT]
3. description: A brief description of the entity based on the text

Return ONLY valid JSON array. Example format:
[
  {{"name": "JOHN SMITH", "type": "PERSON", "description": "CEO of Example Corp who announced the merger"}},
  {{"name": "EXAMPLE CORP", "type": "ORGANIZATION", "description": "Technology company acquiring StartupXYZ"}}
]

TEXT:
{text}

JSON OUTPUT:
"""

RELATIONSHIP_EXTRACTION_PROMPT = """
You are an expert at extracting relationships between entities.

Given the following text and list of entities, extract all relationships between them.
For each relationship provide:
1. source: The source entity name (UPPERCASE)
2. target: The target entity name (UPPERCASE)
3. description: A description of how these entities are related
4. strength: A score from 1-10 indicating relationship strength (10 = very strong)

Return ONLY valid JSON array. Example format:
[
  {{"source": "JOHN SMITH", "target": "EXAMPLE CORP", "description": "John Smith is the CEO of Example Corp", "strength": 9}},
  {{"source": "EXAMPLE CORP", "target": "STARTUPXYZ", "description": "Example Corp is acquiring StartupXYZ", "strength": 8}}
]

ENTITIES:
{entities}

TEXT:
{text}

JSON OUTPUT:
"""

CLAIMS_EXTRACTION_PROMPT = """
You are an expert at extracting factual claims from text.

Extract all specific factual claims from the following text. For each claim provide:
1. subject: The entity the claim is about (UPPERCASE)
2. claim_type: One of [FACT, EVENT, STATEMENT, METRIC, PREDICTION]
3. description: The specific claim or fact
4. date: Associated date/timeframe if mentioned (otherwise empty string)

Focus on:
- Numerical facts (prices, percentages, amounts)
- Events (announcements, launches, decisions)
- Quotes and statements by people
- Predictions and forecasts

Return ONLY valid JSON array. Example format:
[
  {{"subject": "EXAMPLE CORP", "claim_type": "METRIC", "description": "Stock rose 15% in after-hours trading", "date": "2026-02-10"}},
  {{"subject": "JOHN SMITH", "claim_type": "STATEMENT", "description": "Stated that the merger will create 1000 new jobs", "date": ""}}
]

TEXT:
{text}

JSON OUTPUT:
"""


# --- Extraction functions ---

def _parse_llm_json(response: str) -> list:
    """Parse JSON from LLM response, handling markdown code blocks."""
    json_str = response.strip()
    if json_str.startswith("```"):
        json_str = json_str.split("```")[1]
        if json_str.startswith("json"):
            json_str = json_str[4:]
    json_str = json_str.strip()
    return json.loads(json_str)


def extract_entities(text: str, chunk_id: int = 0) -> list[Entity]:
    """Extract entities from a text chunk using the LLM."""
    prompt = ENTITY_EXTRACTION_PROMPT.format(text=text)
    response = chat_ollama(prompt)
    try:
        entities_data = _parse_llm_json(response)
        return [
            Entity(
                name=e.get("name", "").upper(),
                type=e.get("type", "UNKNOWN"),
                description=e.get("description", ""),
                source_chunk=chunk_id
            )
            for e in entities_data
        ]
    except json.JSONDecodeError as ex:
        print(f"[E parse error: {ex}]", end=" ")
        return []


def extract_relationships(text: str, entities: list[Entity], chunk_id: int = 0) -> list[Relationship]:
    """Extract relationships between entities from a text chunk."""
    entity_list = ", ".join([e.name for e in entities])
    prompt = RELATIONSHIP_EXTRACTION_PROMPT.format(text=text, entities=entity_list)
    response = chat_ollama(prompt)
    try:
        rels_data = _parse_llm_json(response)
        return [
            Relationship(
                source=r.get("source", "").upper(),
                target=r.get("target", "").upper(),
                description=r.get("description", ""),
                strength=float(r.get("strength", 5)) / 10.0,
                source_chunk=chunk_id
            )
            for r in rels_data
        ]
    except json.JSONDecodeError as ex:
        print(f"[R parse error: {ex}]", end=" ")
        return []


def extract_claims(text: str, chunk_id: int = 0) -> list[Claim]:
    """Extract factual claims from a text chunk."""
    prompt = CLAIMS_EXTRACTION_PROMPT.format(text=text)
    response = chat_ollama(prompt)
    try:
        claims_data = _parse_llm_json(response)
        return [
            Claim(
                subject=c.get("subject", "").upper(),
                claim_type=c.get("claim_type", "FACT"),
                description=c.get("description", ""),
                date=c.get("date", ""),
                source_chunk=chunk_id
            )
            for c in claims_data
        ]
    except json.JSONDecodeError as ex:
        print(f"[C parse error: {ex}]", end=" ")
        return []


print("Extraction functions defined: extract_entities, extract_relationships, extract_claims")

Extraction functions defined: extract_entities, extract_relationships, extract_claims


In [24]:
# Extract entities, relationships, and claims from all documents
source_results: dict[str, dict] = {}
global_chunk_offset = 0
skipped_chunks: list[dict] = []  # track failures for diagnostics

for doc_idx, doc in enumerate(all_documents):
    doc_chunks = source_chunks[doc.source_id]
    doc_entities: list[Entity] = []
    doc_relationships: list[Relationship] = []
    doc_claims: list[Claim] = []

    print(f"\n{'='*60}")
    print(f"[{doc_idx+1}/{len(all_documents)}] {doc.source_id}: {doc.title[:50]}")
    print(f"  {len(doc_chunks)} chunks to process")

    for i, chunk in enumerate(doc_chunks):
        chunk_id = global_chunk_offset + i
        print(f"  Chunk {i+1}/{len(doc_chunks)}: ", end="")

        try:
            # Entity extraction
            entities = extract_entities(chunk, chunk_id=chunk_id)
            doc_entities.extend(entities)
            print(f"{len(entities)}E ", end="")
            time.sleep(0.5)

            # Relationship extraction
            chunk_entities = [e for e in doc_entities if e.source_chunk == chunk_id]
            if len(chunk_entities) >= 2:
                relationships = extract_relationships(chunk, chunk_entities, chunk_id=chunk_id)
                doc_relationships.extend(relationships)
                print(f"{len(relationships)}R ", end="")
                time.sleep(0.5)
            else:
                print(f"0R ", end="")

            # Claims extraction
            claims = extract_claims(chunk, chunk_id=chunk_id)
            doc_claims.extend(claims)
            print(f"{len(claims)}C")
            time.sleep(0.5)

        except Exception as e:
            err_type = type(e).__name__
            print(f"SKIPPED ({err_type}: {e})")
            skipped_chunks.append({
                "chunk_id": chunk_id,
                "source_id": doc.source_id,
                "chunk_index": i,
                "error": f"{err_type}: {e}",
                "chunk_len": len(chunk),
            })

    global_chunk_offset += len(doc_chunks)

    source_results[doc.source_id] = {
        "entities": doc_entities,
        "relationships": doc_relationships,
        "claims": doc_claims,
        "chunks": doc_chunks,
    }

    print(f"  => {len(doc_entities)}E, {len(doc_relationships)}R, {len(doc_claims)}C")

# Grand totals
total_e = sum(len(r["entities"]) for r in source_results.values())
total_r = sum(len(r["relationships"]) for r in source_results.values())
total_c = sum(len(r["claims"]) for r in source_results.values())
total_processed = sum(len(r["chunks"]) for r in source_results.values())
print(f"\n{'='*60}")
print(f"EXTRACTION COMPLETE")
print(f"Total: {total_e} entities, {total_r} relationships, {total_c} claims")
print(f"Chunks: {total_processed - len(skipped_chunks)} succeeded, {len(skipped_chunks)} skipped")

if skipped_chunks:
    print(f"\n--- Skipped Chunks ---")
    for sc in skipped_chunks:
        print(f"  [{sc['source_id']}] chunk {sc['chunk_index']} ({sc['chunk_len']} chars): {sc['error']}")


[1/1] arxiv:2404.16130: From Local to Global: A Graph RAG Approach to Quer
  193 chunks to process
  Chunk 1/193: 3E 2R 0C
  Chunk 2/193: 9E 5R 0C
  Chunk 3/193: 4E 0R 0C
  Chunk 4/193: 2E 0R 0C
  Chunk 5/193: 3E 0R 0C
  Chunk 6/193: 0E 0R 0C
  Chunk 7/193: 4E 0R 0C
  Chunk 8/193: 0E 0R 0C
  Chunk 9/193: 2E 1R 0C
  Chunk 10/193: 3E 2R 0C
  Chunk 11/193: 1E 0R 0C
  Chunk 12/193: 3E 0R 0C
  Chunk 13/193: 5E 3R 0C
  Chunk 14/193: 4E 11R 0C
  Chunk 15/193: 2E 0R 0C
  Chunk 16/193: 1E 0R 0C
  Chunk 17/193: 9E 0R 0C
  Chunk 18/193: 5E 2R 0C
  Chunk 19/193: 12E 6R 0C
  Chunk 20/193: 7E 0R 0C
  Chunk 21/193: 5E 0R 0C
  Chunk 22/193: 3E 0R 0C
  Chunk 23/193: 2E 0R 0C
  Chunk 24/193: 3E 2R 0C
  Chunk 25/193: 7E 2R 0C
  Chunk 26/193: 3E 0R 0C
  Chunk 27/193: 13E 9R 0C
  Chunk 28/193: 3E 0R 0C
  Chunk 29/193: 3E 1R 0C
  Chunk 30/193: 3E 2R 0C
  Chunk 31/193: 3E 2R 0C
  Chunk 32/193: 0E 0R 0C
  Chunk 33/193: 1E 0R 1C
  Chunk 34/193: 2E 1R 0C
  Chunk 35/193: 4E 3R 0C
  Chunk 36/193: 3E 2R 2C
  Chun

In [25]:
# Display per-source extraction summary
print("=== PER-SOURCE EXTRACTION RESULTS ===\n")
print(f"{'Source':<25} {'Entities':>10} {'Relations':>10} {'Claims':>10}")
print("-" * 60)
for source_id, result in source_results.items():
    print(f"{source_id:<25} {len(result['entities']):>10} {len(result['relationships']):>10} {len(result['claims']):>10}")
print("-" * 60)
print(f"{'TOTAL':<25} {total_e:>10} {total_r:>10} {total_c:>10}")

=== PER-SOURCE EXTRACTION RESULTS ===

Source                      Entities  Relations     Claims
------------------------------------------------------------
arxiv:2404.16130                 623        198          5
------------------------------------------------------------
TOTAL                            623        198          5


In [26]:
# Deduplicate entities by name (merge descriptions)
def deduplicate_entities(entities: list[Entity]) -> list[Entity]:
    """Merge duplicate entities, combining their descriptions."""
    entity_map: dict[str, Entity] = {}
    
    for entity in entities:
        key = entity.name
        if key in entity_map:
            existing = entity_map[key]
            if entity.description and entity.description not in existing.description:
                existing.description = f"{existing.description} | {entity.description}"
        else:
            entity_map[key] = Entity(
                name=entity.name,
                type=entity.type,
                description=entity.description,
                source_chunk=entity.source_chunk
            )
    
    return list(entity_map.values())


def merge_entities_across_sources(
    source_results: dict[str, dict]
) -> tuple[list[Entity], list[Relationship], list[Claim]]:
    """Deduplicate within each source, then merge across all sources.
    
    Returns global_entities, global_relationships, global_claims.
    Also builds entity_source_map: entity_name -> list of source_ids.
    """
    # Phase 1: Deduplicate within each source
    for source_id, result in source_results.items():
        result["unique_entities"] = deduplicate_entities(result["entities"])
    
    # Phase 2: Merge across all sources
    global_entity_map: dict[str, Entity] = {}
    entity_source_map: dict[str, list[str]] = {}
    
    for source_id, result in source_results.items():
        for entity in result["unique_entities"]:
            key = entity.name
            if key in global_entity_map:
                existing = global_entity_map[key]
                if entity.description and entity.description not in existing.description:
                    existing.description = f"{existing.description} | {entity.description}"
                entity_source_map[key].append(source_id)
            else:
                global_entity_map[key] = Entity(
                    name=entity.name,
                    type=entity.type,
                    description=entity.description,
                    source_chunk=entity.source_chunk
                )
                entity_source_map[key] = [source_id]
    
    global_entities = list(global_entity_map.values())
    
    # Merge all relationships (keep all, some may reference same entities)
    global_relationships = []
    for result in source_results.values():
        global_relationships.extend(result["relationships"])
    
    # Merge all claims
    global_claims = []
    for result in source_results.values():
        global_claims.extend(result["claims"])
    
    return global_entities, global_relationships, global_claims, entity_source_map


# Run cross-document merge
global_entities, global_relationships, global_claims, entity_source_map = merge_entities_across_sources(source_results)

# Build entity -> chunk provenance mapping from raw extraction data.
# Each entity was extracted from a specific chunk; this records ALL chunks
# that produced each entity (before deduplication collapsed them).
entity_chunk_map: dict[str, list[dict]] = {}
for doc in all_documents:
    result = source_results[doc.source_id]
    for entity in result["entities"]:
        key = entity.name
        if key not in entity_chunk_map:
            entity_chunk_map[key] = []
        entry = {"chunk_index": entity.source_chunk, "source_id": doc.source_id}
        if entry not in entity_chunk_map[key]:
            entity_chunk_map[key].append(entry)

# Per-source dedup stats
print("=== PER-SOURCE DEDUPLICATION ===\n")
for source_id, result in source_results.items():
    raw = len(result["entities"])
    unique = len(result["unique_entities"])
    print(f"  {source_id}: {raw} raw -> {unique} unique")

print(f"\n=== GLOBAL MERGE ===")
print(f"  Global unique entities: {len(global_entities)}")
print(f"  Global relationships: {len(global_relationships)}")
print(f"  Global claims: {len(global_claims)}")

# Chunk provenance stats
entities_with_chunks = sum(1 for v in entity_chunk_map.values() if v)
multi_chunk = sum(1 for v in entity_chunk_map.values() if len(v) > 1)
print(f"\n=== CHUNK PROVENANCE ===")
print(f"  Entities with chunk tracking: {entities_with_chunks}")
print(f"  Entities in 2+ chunks: {multi_chunk}")

=== PER-SOURCE DEDUPLICATION ===

  arxiv:2404.16130: 623 raw -> 466 unique

=== GLOBAL MERGE ===
  Global unique entities: 466
  Global relationships: 198
  Global claims: 5

=== CHUNK PROVENANCE ===
  Entities with chunk tracking: 466
  Entities in 2+ chunks: 69


## Summary: Extraction Results

Consolidate all extracted knowledge elements.

In [27]:
print("="*60)
print("GRAPHRAG MULTI-SOURCE EXTRACTION SUMMARY")
print("="*60)
print(f"\nSources: {len(all_documents)}")
print(f"Total content: {total_chars:,} characters")
print(f"Total chunks: {total_chunks}")
print(f"\nGlobal entities: {len(global_entities)}")
print(f"Global relationships: {len(global_relationships)}")
print(f"Global claims: {len(global_claims)}")

# Entity type breakdown
print("\n--- Entity Types ---")
type_counts: dict[str, int] = {}
for e in global_entities:
    type_counts[e.type] = type_counts.get(e.type, 0) + 1
for t, count in sorted(type_counts.items(), key=lambda x: -x[1]):
    print(f"  {t}: {count}")

# Claim type breakdown
print("\n--- Claim Types ---")
claim_type_counts: dict[str, int] = {}
for c in global_claims:
    claim_type_counts[c.claim_type] = claim_type_counts.get(c.claim_type, 0) + 1
for t, count in sorted(claim_type_counts.items(), key=lambda x: -x[1]):
    print(f"  {t}: {count}")

# Cross-source entities
multi_source = {k: v for k, v in entity_source_map.items() if len(v) > 1}
print(f"\n--- Cross-Source Entities ({len(multi_source)} entities in 2+ sources) ---")
for name, sources in sorted(multi_source.items(), key=lambda x: -len(x[1])):
    print(f"  {name}: {sources}")

GRAPHRAG MULTI-SOURCE EXTRACTION SUMMARY

Sources: 1
Total content: 89,608 characters
Total chunks: 193

Global entities: 466
Global relationships: 198
Global claims: 5

--- Entity Types ---
  PERSON: 258
  CONCEPT: 81
  ORGANIZATION: 32
  EVENT: 30
  PRODUCT: 30
  LOCATION: 14
  DATE: 9
  MONEY: 5
  JOURNAL: 3
  PROJECT: 1
  SOFTWARE: 1
  ELEMENT_INSTANCE: 1
  : 1

--- Claim Types ---
  EVENT: 3
  FACT: 2

--- Cross-Source Entities (0 entities in 2+ sources) ---


## Next Steps

In the next notebook we will:
1. **Build the knowledge graph** - Store entities and relationships from all 7 sources in a graph structure
2. **Apply community detection** - Use Louvain algorithm to find cross-domain topic clusters
3. **Generate community summaries** - Create hierarchical summaries for each cluster
4. **Interactive visualization** - Explore the graph with ipycytoscape
5. **Store in SQLite** - Persist the graph for retrieval

In [28]:
# Export extracted data in multi-document format
extraction_results = {
    "sources": [],
    "merged": {
        "entities": [{"name": e.name, "type": e.type, "description": e.description} for e in global_entities],
        "relationships": [{"source": r.source, "target": r.target, "description": r.description, "strength": r.strength} for r in global_relationships],
        "claims": [{"subject": c.subject, "claim_type": c.claim_type, "description": c.description, "date": c.date} for c in global_claims],
        "entity_source_map": {k: v for k, v in entity_source_map.items()},
        "entity_chunk_map": entity_chunk_map,
        "total_chunks": total_chunks,
        "total_sources": len(all_documents),
    }
}

# Add per-source data
for doc in all_documents:
    result = source_results[doc.source_id]
    extraction_results["sources"].append({
        "source_id": doc.source_id,
        "source_type": doc.source_type,
        "title": doc.title,
        "url": doc.url,
        "content_type": doc.content_type,
        "content_length": len(doc.content),
        "fetched_at": doc.fetched_at,
        "chunks": result["chunks"],
        "entities": [{"name": e.name, "type": e.type, "description": e.description} for e in result.get("unique_entities", result["entities"])],
        "relationships": [{"source": r.source, "target": r.target, "description": r.description, "strength": r.strength} for r in result["relationships"]],
        "claims": [{"subject": c.subject, "claim_type": c.claim_type, "description": c.description, "date": c.date} for c in result["claims"]],
    })

with open("extraction_results.json", "w") as f:
    json.dump(extraction_results, f, indent=2)

print(f"Results saved to extraction_results.json")
print(f"  {len(extraction_results['sources'])} sources")
print(f"  {len(extraction_results['merged']['entities'])} merged entities")
print(f"  {len(extraction_results['merged']['relationships'])} relationships")
print(f"  {len(extraction_results['merged']['claims'])} claims")
print(f"  {len(extraction_results['merged']['entity_chunk_map'])} entities with chunk provenance")

Results saved to extraction_results.json
  1 sources
  466 merged entities
  198 relationships
  5 claims
  466 entities with chunk provenance


In [29]:
# Cross-source entity overlap analysis
multi_source = {k: v for k, v in entity_source_map.items() if len(v) > 1}

print("="*60)
print(f"CROSS-SOURCE ENTITY OVERLAP: {len(multi_source)} entities in 2+ sources")
print("="*60)

if multi_source:
    for name, sources in sorted(multi_source.items(), key=lambda x: -len(x[1])):
        entity = next((e for e in global_entities if e.name == name), None)
        etype = entity.type if entity else "?"
        print(f"\n  [{etype}] {name} (in {len(sources)} sources)")
        for sid in sources:
            doc = next((d for d in all_documents if d.source_id == sid), None)
            if doc:
                print(f"    - {sid}: {doc.title[:50]}")
else:
    print("\n  No cross-source entities found.")
    print("  This is expected with diverse topics — entities are domain-specific.")

# Source overlap matrix
print(f"\n{'='*60}")
print("SOURCE PAIR OVERLAP (shared entity count)")
print("="*60)
source_ids = [doc.source_id for doc in all_documents]
for i, s1 in enumerate(source_ids):
    for s2 in source_ids[i+1:]:
        shared = [name for name, srcs in entity_source_map.items()
                  if s1 in srcs and s2 in srcs]
        if shared:
            print(f"  {s1} <-> {s2}: {len(shared)} ({', '.join(shared[:5])}{'...' if len(shared) > 5 else ''})")

CROSS-SOURCE ENTITY OVERLAP: 0 entities in 2+ sources

  No cross-source entities found.
  This is expected with diverse topics — entities are domain-specific.

SOURCE PAIR OVERLAP (shared entity count)
